In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()  # Upload `kaggle.json`

KeyboardInterrupt: 

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d anzarusmani/monuments

In [ ]:
!unzip -q monuments.zip -d archive

In [ ]:
!ls archive

In [ ]:
!ls archive/Monument/

In [ ]:
!ls archive/Monument/Monument/

In [ ]:
import cv2
import numpy as np
from PIL import Image
from noise import pnoise2  # pip install noise
import random
import os

# ==================================================================
# Damage Generation Functions
# ==================================================================

def generate_cracks_mask(shape, scale=20.0, threshold=0.4):
    """Generate fractal cracks using Perlin noise"""
    height, width = shape
    mask = np.zeros((height, width), dtype=np.float32)

    for y in range(height):
        for x in range(width):
            nx = x/scale
            ny = y/scale
            noise_val = pnoise2(nx, ny, octaves=6)
            if noise_val > threshold:
                mask[y,x] = 1.0
    return cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3,3)))

def add_scratches(mask, num_scratches=5):
    """Add curved scratches with random thickness"""
    height, width = mask.shape
    for _ in range(num_scratches):
        x1, y1 = random.randint(0, width), random.randint(0, height)
        angle = random.uniform(0, 2*np.pi)
        length = random.randint(20, 100)
        thickness = random.randint(1, 3)

        # Create curved scratch using Bézier curve
        x2 = int(x1 + length * np.cos(angle))
        y2 = int(y1 + length * np.sin(angle))
        control_x = int((x1 + x2)/2 + random.randint(-30,30))
        control_y = int((y1 + y2)/2 + random.randint(-30,30))

        pts = np.array([[x1,y1], [control_x,control_y], [x2,y2]], np.int32)
        pts = pts.reshape((-1,1,2))
        cv2.polylines(mask, [pts], False, 1.0, thickness)
    return mask

def create_peeling_effect(mask):
    """Simulate paint peeling using morphological operations"""
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)
    return cv2.GaussianBlur(mask, (5,5), 0)

def generate_stain_mask(shape):
    """Create organic-looking stains using Perlin noise + circles"""
    height, width = shape
    mask = np.zeros((height, width))

    # Generate Perlin noise base
    for y in range(height):
        for x in range(width):
            nx = x/50.0
            ny = y/50.0
            noise_val = pnoise2(nx, ny, octaves=3)
            if noise_val > 0.3:
                mask[y,x] = min(mask[y,x] + 0.5, 1.0)

    # Add circular stains
    for _ in range(random.randint(3,7)):
        center = (random.randint(0, width), random.randint(0, height))
        radius = random.randint(10, 40))
        cv2.circle(mask, center, radius, 1.0, -1)

    return cv2.GaussianBlur(mask, (15,15), 0)

# ==================================================================
# Main Damage Application Pipeline
# ==================================================================

def apply_realistic_damage(image_path, output_dir):
    # Load original image
    orig_img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    h, w = orig_img.shape[:2]

    # Initialize composite mask
    damage_mask = np.zeros((h, w), dtype=np.float32)

    # Apply cracks (40% probability)
    if random.random() < 0.4:
        crack_mask = generate_cracks_mask((h, w), scale=15.0)
        damage_mask = np.maximum(damage_mask, crack_mask)

    # Apply scratches (60% probability)
    if random.random() < 0.6:
        scratch_mask = np.zeros_like(damage_mask)
        scratch_mask = add_scratches(scratch_mask, num_scratches=random.randint(3,8))
        damage_mask = np.maximum(damage_mask, scratch_mask)

    # Apply peeling (30% probability)
    if random.random() < 0.3:
        peeling_mask = create_peeling_effect(np.random.rand(h,w) > 0.9).astype(np.float32))
        damage_mask = np.maximum(damage_mask, peeling_mask)

    # Apply stains (50% probability)
    if random.random() < 0.5:
        stain_mask = generate_stain_mask((h, w))
        damage_mask = np.maximum(damage_mask, stain_mask)

    # Normalize and threshold mask
    damage_mask = np.clip(damage_mask, 0, 1)
    damage_mask = (damage_mask > 0.3).astype(np.uint8)

    # Apply damage to image
    damaged_img = orig_img.copy()

    # 1. Color fading in damaged areas
    damaged_img = (damaged_img * (1 - damage_mask[..., np.newaxis] * 0.7)).astype(np.uint8)

    # 2. Add texture to damaged areas
    noise = np.random.randint(0, 60, (h, w, 3), dtype=np.uint8)
    damaged_img = cv2.addWeighted(damaged_img, 1,
                                 noise, 0.3,
                                 gamma=0,
                                 mask=damage_mask)

    # Save results
    base_name = os.path.basename(image_path)
    Image.fromarray(damaged_img).save(os.path.join(output_dir, f"damaged_{base_name}"))
    Image.fromarray(damage_mask * 255).save(os.path.join(output_dir, f"mask_{base_name}"))

# ==================================================================
# Usage Example
# ==================================================================

if __name__ == "__main__":
    # Configure paths
    input_dir = "path/to/clean/images"
    output_dir = "path/to/damaged/images"
    os.makedirs(output_dir, exist_ok=True)

    # Process all images
    for img_file in os.listdir(input_dir):
        if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(input_dir, img_file)
            apply_realistic_damage(img_path, output_dir)

    print(f"Generated {len(os.listdir(output_dir))//2} damaged images with masks")